In [1]:
import os
from pathlib import Path
import subprocess
import math
import random
from collections import defaultdict
import posixpath
import glob

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [149]:
from util import print_pd_table_for_libreoffice

ModuleNotFoundError: No module named 'util'

# Batch Analysis

In [129]:
batch_paths = glob.glob("../batches/mine/**/*.npz")
batch_paths

['../batches/mine/train/batch_0512-0768.npz',
 '../batches/mine/train/batch_1280-1536.npz',
 '../batches/mine/train/batch_5632-5888.npz',
 '../batches/mine/train/batch_4352-4608.npz',
 '../batches/mine/train/batch_4096-4352.npz',
 '../batches/mine/train/batch_1024-1280.npz',
 '../batches/mine/train/batch_1536-1792.npz',
 '../batches/mine/train/batch_3840-4096.npz',
 '../batches/mine/train/batch_4608-4864.npz',
 '../batches/mine/train/batch_7168-7424.npz',
 '../batches/mine/train/batch_6144-6400.npz',
 '../batches/mine/train/batch_2560-2816.npz',
 '../batches/mine/train/batch_3328-3584.npz',
 '../batches/mine/train/batch_8192-8448.npz',
 '../batches/mine/train/batch_0000-0256.npz',
 '../batches/mine/train/batch_4864-5120.npz',
 '../batches/mine/train/batch_6656-6912.npz',
 '../batches/mine/train/batch_5120-5376.npz',
 '../batches/mine/train/batch_8960-9216.npz',
 '../batches/mine/train/batch_7936-8192.npz',
 '../batches/mine/train/batch_7680-7936.npz',
 '../batches/mine/train/batch_2304

In [130]:
def get_dset_group(path):
    path = path.replace("\\", "/")
    if "/train/" in path:
        return "train"
    elif "/test/" in path:
        return "test"
    elif "/validation/" in path:
        return "validation"
    

In [131]:
a = np.linspace(1, 9, 9).reshape((3, 3))
np.maximum(a, 3)

array([[3., 3., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

In [132]:
start_locs = defaultdict(lambda: np.zeros((0, 3)))
end_locs = defaultdict(lambda: np.zeros((0, 3)))
dists_travelled_motions = defaultdict(lambda: np.zeros((0,)))
dists_travelled_windows = defaultdict(lambda: np.zeros((0,)))
dists_travelled_frames = defaultdict(lambda: np.zeros((0,)))

ws = 24

for batch_fp in batch_paths:
    grp = get_dset_group(batch_fp)
    with np.load(batch_fp) as bb:
        dtransl = bb['dtransls']
        nframes = bb['nframes']
        transl = np.cumsum(dtransl, axis=1)
        nb = dtransl.shape[0]
        nbrng = np.array(range(nb))
        
        start_cur = transl[:, 0]
        end_cur = transl[nbrng, nframes - 1]
        dist_cur = np.linalg.norm(end_cur - start_cur, axis=-1)
        
        start_locs[grp] = np.concatenate([start_locs[grp], start_cur])
        end_locs[grp] = np.concatenate([end_locs[grp], end_cur])
        dists_travelled_motions[grp] = np.concatenate([dists_travelled_motions[grp], dist_cur])
        
        rand = np.random.random((nb, 1))
        wstart_inds = (np.maximum(0, nframes - 1 - ws)[:, np.newaxis] * rand).astype(int).flatten()
        wend_inds = wstart_inds + ws
        
        #print(wstart_inds)
        #print(nframes)
        
        dist_w_cur = np.linalg.norm(transl[nbrng, wend_inds] - transl[nbrng, wstart_inds], axis=-1)
        dists_travelled_windows[grp] = np.concatenate([dists_travelled_windows[grp], dist_w_cur])
        
        dist_frm_cur = np.linalg.norm(dtransl[nbrng, wstart_inds], axis=-1)
        dists_travelled_frames[grp] = np.concatenate([dists_travelled_frames[grp], dist_frm_cur])

In [133]:
start_locs['train'][:10]

array([[ 0.2149666 ,  0.86172737, -0.02437929],
       [ 0.22535357,  0.8515224 , -0.03916353],
       [ 1.13859597,  0.98743956,  1.21851803],
       [ 0.4499025 ,  0.94938421,  1.18308114],
       [ 0.25050154,  0.9421374 ,  0.03439322],
       [ 0.53921013,  0.79426941,  0.95166352],
       [ 0.2232168 ,  0.93248259, -0.00209716],
       [ 0.25033186,  0.91657399,  0.0118898 ],
       [ 0.23449883,  0.84288445,  0.00343627],
       [ 0.24865984,  0.83967831, -0.01231338]])

In [134]:
end_locs['train'][:10]

array([[ 0.24295877,  0.88296946,  0.01098711],
       [ 0.21529266,  0.8824534 , -0.00727721],
       [ 1.2362341 ,  0.94390263, -0.95770271],
       [ 0.55054514,  0.92918398, -0.98915385],
       [ 0.2308685 ,  1.00508446,  0.02702811],
       [ 0.54633012,  0.79071576,  0.96729195],
       [ 0.21276463,  0.89953041,  0.02281663],
       [ 0.24584423,  0.92154026,  0.0350648 ],
       [ 0.24955566,  0.87820306,  0.00267481],
       [ 0.22430513,  0.93942037, -0.0219228 ]])

In [135]:
dists_travelled_motions['train'][-10:]

array([2.53208801, 3.54542364, 2.2681921 , 3.58164043, 3.54128259,
       0.10067275, 0.0675425 , 0.03632084, 0.03238085, 0.02894134])

In [136]:
dists_travelled_windows['train'][-10:]

array([0.99606428, 1.5047433 , 0.89790716, 1.62633451, 2.54810696,
       0.02044783, 0.02378718, 0.05471274, 0.01970734, 0.05018923])

In [137]:
dists_travelled_frames['train'][-10:]

array([0.03622381, 0.01611381, 0.03507539, 0.05166196, 0.05507313,
       0.01906069, 0.93497848, 0.01661905, 0.0209562 , 0.02052191])

In [138]:
dtransls[30,10]

array([ 0.07677509, -0.00179388, -0.10298842])

## Analyse Global Motion

In [146]:
for dset in start_locs.keys():
    stlocs = start_locs[dset]
    edlocs = end_locs[dset]
    df = pd.DataFrame(data={
        'start_locs_x': stlocs[:, 0],
        'start_locs_y': stlocs[:, 1],
        'start_locs_z': stlocs[:, 2],
        'end_locs_x': edlocs[:, 0],
        'end_locs_y': edlocs[:, 1],
        'end_locs_z': edlocs[:, 2],
        'dists_total': dists_travelled_motions[dset],
        'dists_wind': dists_travelled_windows[dset],
        'dists_frames': dists_travelled_frames[dset]
    })
    print(dset)
    pd.options.display.float_format = '{:.2f}'.format
    #display(df.describe())
    dfstr = df.describe().to_string()
    while "  " in dfstr:
        dfstr = dfstr.replace("  ", " ")
    dfstr = dfstr.replace(" ", "\t")
    print(dfstr)

train
	start_locs_x	start_locs_y	start_locs_z	end_locs_x	end_locs_y	end_locs_z	dists_total	dists_wind	dists_frames
count	9472.00	9472.00	9472.00	9472.00	9472.00	9472.00	9472.00	9472.00	9472.00
mean	0.38	0.90	0.35	0.42	0.90	0.09	0.92	0.29	0.03
std	0.55	0.11	1.10	0.56	0.12	1.16	1.26	0.42	0.16
min	-2.00	0.03	-2.91	-3.44	0.09	-3.04	0.00	0.00	0.00
25%	0.22	0.86	-0.10	0.22	0.86	-0.50	0.04	0.02	0.00
50%	0.33	0.90	0.04	0.36	0.90	-0.00	0.21	0.10	0.01
75%	0.62	0.93	1.00	0.65	0.94	0.42	1.63	0.44	0.02
max	3.14	1.80	3.97	4.31	1.80	3.92	6.47	3.89	3.65
test
	start_locs_x	start_locs_y	start_locs_z	end_locs_x	end_locs_y	end_locs_z	dists_total	dists_wind	dists_frames
count	1280.00	1280.00	1280.00	1280.00	1280.00	1280.00	1280.00	1280.00	1280.00
mean	0.09	0.80	-0.18	0.27	0.80	-0.25	0.71	0.30	0.03
std	0.84	0.20	0.81	0.84	0.20	0.89	1.85	0.48	0.18
min	-3.67	0.05	-4.09	-3.80	0.05	-4.18	0.00	0.00	0.00
25%	0.11	0.83	-0.40	0.11	0.83	-0.40	0.03	0.02	0.00
50%	0.16	0.86	-0.37	0.18	0.86	-0.37	0.05	0.11	0.01
75%	0.24

# Individual Analysis

In [7]:
qd1_path = "../batches/qualitative/ACCAD/Male2Running_c3d/C1 - stand to run_poses.npz"
qd2_path = "../batches/qualitative/DFaust/50009/50009_jumping_jacks_poses.npz"

In [11]:
qd1 = np.load(qd1_path)
qd2 = np.load(qd2_path)
[k for k in qd1.keys()]

['trans', 'gender', 'mocap_framerate', 'betas', 'dmpls', 'poses']

In [12]:
print('qd1 len:', qd1['trans'].shape[0])
print('qd2 len:', qd2['trans'].shape[0])

qd1 len: 491
qd2 len: 190


In [15]:
MAXFRMS = 20
print('qd1 trans up:', qd1['trans'][:MAXFRMS, 2])
print('qd2 trans up:', qd2['trans'][:MAXFRMS, 2])

qd1 trans up: [0.91671033 0.91660728 0.9164975  0.91640931 0.91635995 0.91447156
 0.91410678 0.91388859 0.91374815 0.91362353 0.91354771 0.91355957
 0.91362308 0.9135671  0.91343905 0.91333347 0.91330151 0.91332011
 0.91332429 0.91334287]
qd2 trans up: [0.36650758 0.36643808 0.36647634 0.36675986 0.36734945 0.3681767
 0.36954815 0.37133744 0.37335314 0.37536155 0.3771561  0.37842043
 0.37917077 0.37925049 0.3787719  0.37790543 0.37648232 0.37465318
 0.37228812 0.36925029]
